# UCI Heart Diseaseデータセット

1. age（年齢）
    - 患者の年齢（単位：年）

2. sex（性別）
    - 1：男性
    - 0：女性

3. cp（胸痛の種類：Chest Pain Type）
    - 患者が経験した胸痛のタイプ
    - 0：無症状（典型的狭心症ではない）
    - 1：狭心症様の胸痛（典型的な狭心症）
    - 2：非狭心症様の胸痛（非典型的な狭心症）
    - 3：非心臓性胸痛（心臓に起因しない胸痛）

4. trestbps（安静時血圧：Resting Blood Pressure）
    - 入院時の安静時の収縮期血圧（単位：mm Hg）

5. chol（コレステロール値：Serum Cholesterol）
    - 血清コレステロール値（単位：mg/dl）

6. fbs（空腹時血糖値：Fasting Blood Sugar）
    - 空腹時血糖値が120 mg/dlを超えるかどうか
    - 1：空腹時血糖値 > 120 mg/dl（糖尿病の疑いあり）
    - 0：空腹時血糖値 ≤ 120 mg/dl

7. restecg（安静時心電図結果：Resting Electrocardiographic Results）
    - 安静時の心電図結果
    - 0：正常
    - 1：ST-T波異常（例：逆転や高振幅のST-T波）
    - 2：左心室肥大の可能性を示す確定的な心電図所見

8. thalach（最大心拍数：Maximum Heart Rate Achieved）
    - 運動中に達成された最大心拍数

9. exang（運動誘発性狭心症：Exercise-Induced Angina）
    - 運動によって狭心症が誘発されたかどうか
    - 1：はい（狭心症が誘発された）
    - 0：いいえ（狭心症は誘発されなかった）

10. oldpeak（ST低下度：ST Depression Induced by Exercise Relative to Rest）
    - 運動によるSTセグメントの低下量（単位：mm）
    - 安静時と比較した運動時のSTセグメントの低下度合いを示します。

11. slope（ピーク運動時のSTセグメントの傾き：Slope of the Peak Exercise ST Segment）
    - 運動時のSTセグメントの傾き
    - 0：下向き（ダウンスロープ）
    - 1：平坦（フラット）
    - 2：上向き（アップスロープ）

12. ca（フルオロスコピーで着色された主要血管の数：Number of Major Vessels Colored by Fluoroscopy）
    - フルオロスコピー検査で可視化された主要な心臓血管の数（0から3の整数）

13. thal（サラセミアの状態：Thalassemia）
    - サラセミア（血液疾患）の種類
    - 3：正常（ノーマル）
    - 6：固定欠損（フィックスド・ディフェクト）
    - 7：可逆的欠損（リバーシブル・ディフェクト）

14. num（診断結果：Diagnosis of Heart Disease）
    - 心疾患の診断結果
    - 0：心疾患なし（< 50% の血管狭窄）
    - 1〜4：心疾患あり（数値が大きいほど疾患の程度が深刻）
    - 1：1本の血管に>50%の狭窄
    - 2：2本の血管に>50%の狭窄
    - 3：3本の血管に>50%の狭窄
    - 4：左主幹部の狭窄または4本以上の血管に狭窄

## データの読み込み

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 列名を定義
column_names = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
    'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
    'ca', 'thal', 'num'
]

# データセットの読み込み
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
df = pd.read_csv(url, names=column_names)

# データの前処理
df.replace('?', np.nan, inplace=True)  # '?'をNaNに置換
df.dropna(inplace=True)  # 欠損値を含む行を削除
df = df.apply(pd.to_numeric)  # すべてのデータを数値型に変換

# ターゲット変数を二値化（0：疾患なし、1：疾患あり）
df['num'] = df['num'].apply(lambda x: 1 if x > 0 else 0)

df

## グラフによるデータの確認

In [ ]:
x_column = 'age'
y_column = 'trestbps'

fig, ax = plt.subplots()
ax.scatter(df[df['num']==0][x_column], df[df['num']==0][y_column], label='no heart disease')
ax.scatter(df[df['num']==1][x_column], df[df['num']==1][y_column], label='heart disease')
ax.legend()

ax.set_xlabel(x_column)
ax.set_ylabel(y_column)
plt.show()

## データベースを学習用とテスト用に分割

In [ ]:
# 特徴量とターゲットに分割
X = df.drop('num', axis=1).values
y = df['num'].values

# データを学習用とテスト用に分割
def train_test_split(X, y, test_size=0.2, random_state=None):
    np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    test_size = int(X.shape[0] * test_size)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## フィッシャーの線形判別

In [ ]:
# クラスごとの平均ベクトルを計算
mean_0 = np.mean(X_train[y_train == 0], axis=0)
mean_1 = np.mean(X_train[y_train == 1], axis=0)
mean_diff = mean_0 - mean_1

# クラス内散布行列（Sw）を計算
S_w = np.zeros((X_train.shape[1], X_train.shape[1]))
for c, mean in zip([0, 1], [mean_0, mean_1]):
    X_c = X_train[y_train == c]
    S_w += np.dot((X_c - mean).T, (X_c - mean))

# 投影ベクトルwを計算
w = np.linalg.inv(S_w).dot(mean_diff)

# データを新しい軸に投影
X_train_lda = X_train.dot(w)
X_test_lda = X_test.dot(w)

# クラスごとの投影後の平均を計算
mean_lda_0 = np.mean(X_train_lda[y_train == 0])
mean_lda_1 = np.mean(X_train_lda[y_train == 1])

# 閾値をクラス間の平均とする
threshold = (mean_lda_0 + mean_lda_1) / 2

# 予測を行う
y_pred = np.where(X_test_lda > threshold, 0, 1)  # 投影値が閾値より大きければクラス0、小さければクラス1

# 混同行列を計算
def confusion_matrix(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    return np.array([[tn, fp],
                     [fn, tp]])

cm = confusion_matrix(y_test, y_pred)
print("混同行列:")
print(cm)

# 精度、再現率、F1スコアを計算
def classification_report(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp = cm[0]
    fn, tp = cm[1]
    precision_0 = tn / (tn + fn) if (tn + fn) > 0 else 0
    recall_0 = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1_score_0 = 2 * precision_0 * recall_0 / (precision_0 + recall_0) if (precision_0 + recall_0) > 0 else 0

    precision_1 = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall_1 = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score_1 = 2 * precision_1 * recall_1 / (precision_1 + recall_1) if (precision_1 + recall_1) > 0 else 0

    print("\n分類レポート:")
    print(f"クラス 0 - 適合率: {precision_0:.2f}, 再現率: {recall_0:.2f}, F1スコア: {f1_score_0:.2f}")
    print(f"クラス 1 - 適合率: {precision_1:.2f}, 再現率: {recall_1:.2f}, F1スコア: {f1_score_1:.2f}")
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    print(f"\n全体の正解率: {accuracy:.2f}")

classification_report(y_test, y_pred)
